## LR Scraper and Estimator

Overall, this notebook contains code to scrape diagnostic likelihood ratios from theNNT.com and convert them to numerical form. 

It also contains code to generate prompts for large language models to estimate the likelihood ratios. 

The output is a spreadsheet called: nnt_lrs_with_estimated which contains: 
- a sheet for each diagnosis or prediction target
- a row for each piece of information
- columns for the name, raw nnt lr, processed nnt lr, and estimated by 1 or more LLMs

### Extract Data from the NNT 

This scrapes all of the likelihood ratios listed on the NNT ('https://thennt.com/home-lr/') into Excel spreadsheets. 

1. A spreadsheet ("nnt_lrs.xlsx") contains a separate sheet for each page, which corresponds to a "prediction tasks" e.g. diagnosing the cause of a symptom - sometimes with specification of an intended population. Each sheet contains two columns: the name of the features (e.g. test result, finding, historical occurence, comorbditiy), the second contains the raw listing from the spreadsheet

2. A second spreadsheet contains the same sheets corresponding to a prediction target, and all of the features. These are used two call the 

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
import math
import os
import json
import pandas as pd
from pydantic import BaseModel
from openai import OpenAI  # or your appropriate client wrapper
import math
from dotenv import load_dotenv

load_dotenv()  # looks for a .env file in the current dir by default
#print(os.getenv("OPENAI_API_KEY"))


False

In [ ]:
def get_specialty_links():
    """
    Extracts specialties and their corresponding article links from the webpage.
    Returns a list of dictionaries with specialty names and associated links.
    """

    url = 'https://thennt.com/home-lr/'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the section with "Diagnosis (LR) Reviews by Specialty"
    specialty_section = soup.find('div', class_='well subdisplay accordion_caption', id='lr-byspecialty')

    if not specialty_section:
        print("Could not find the 'Diagnosis (LR) Reviews by Specialty' section on the webpage.")
        return []

    results = []

    # Find all specialty headings (e.g., h3)
    subheadings = specialty_section.find_all('h3')

    for subheading in subheadings:
        subheading_text = subheading.get_text(strip=True)  # Get specialty name
        links = []

        # Find the next unordered list (ul) which contains links
        next_ul = subheading.find_next_sibling('ul')

        if next_ul:
            for a_tag in next_ul.find_all('a', href=True):
                link_text = a_tag.get_text(strip=True)  # Link display name
                link_href = a_tag['href']  # Actual URL
                links.append({'display_name': link_text, 'url': link_href})

        results.append({'specialty': subheading_text, 'links': links})

    return results

def extract_likelihood_ratios(page_content):
    """
    Parses all likelihood ratio tables within <article class="lr_cards_details">.
    Extracts findings and their likelihood ratios, ensuring sequential <td> pairs are handled correctly.
    Returns a list of tuples: (finding, likelihood ratio).
    """
    soup = BeautifulSoup(page_content, 'html.parser')
    results = []

    # Locate the section containing likelihood ratio tables
    lr_section = soup.find('article', class_='lr_cards_details')
    if not lr_section:
        return results  # Return empty if no section found

    # Find all tables inside the LR card
    tables = lr_section.find_all('table', class_='lrtable')

    for table in tables:
        # Grab all <tr> elements
        all_rows = table.find_all('tr')

        # Filter out any row that only has <th> (i.e., a header row)
        data_rows = []
        for row in all_rows:
            # If there's at least one <td> in this row, treat it as a data row
            if row.find_all('td'):
                data_rows.append(row)

        # If we have real data rows, parse them
        if data_rows:
            for row in data_rows:
                cols = row.find_all('td')
                # If the row has exactly 2 <td>, treat them as (finding, LR)
                if len(cols) == 2:
                    finding = cols[0].get_text(strip=True)
                    lr_value = cols[1].get_text(strip=True)
                    # If there's an <a> inside the LR cell, grab its text
                    link = cols[1].find('a')
                    if link:
                        lr_value = link.get_text(strip=True) or lr_value
                    if not lr_value:
                        lr_value = "Not reported"

                    results.append((finding, lr_value))

        else:
            # Fallback: if there are no valid data rows, we process all <td> in pairs
            cols = table.find_all('td')
            for i in range(0, len(cols) - 1, 2):
                finding = cols[i].get_text(strip=True)
                lr_value_element = cols[i + 1]

                # Extract the likelihood ratio, handling nested <a> and <br/>
                link = lr_value_element.find('a')
                if link:
                    lr_value = link.get_text(strip=True)
                else:
                    lr_value = lr_value_element.get_text(strip=True)

                if not lr_value:
                    lr_value = "Not reported"

                results.append((finding, lr_value))

    return results

def fetch_webpages(specialty_links):
    """
    Iterates through all the extracted links, fetches the webpage content, 
    and extracts likelihood ratio findings.
    """
    findings_by_display_name = {}

    for item in specialty_links:
        print(f"Fetching pages for Specialty: {item['specialty']}")

        for link in item['links']:
            display_name = link['display_name']
            url = link['url']

            try:
                print(f"  - Fetching: {display_name} ({url})")
                response = requests.get(url)

                if response.status_code == 200:
                    print(f"    Success: {display_name} page fetched.")
                    
                    # Extract likelihood ratio findings
                    findings = extract_likelihood_ratios(response.text)
                    
                    # Store the extracted data
                    findings_by_display_name[display_name] = findings

                else:
                    print(f"    Failed to fetch {display_name} - Status Code: {response.status_code}")

                time.sleep(1)  # Optional: Add a delay to avoid overwhelming the server

            except requests.RequestException as e:
                print(f"    Error fetching {display_name}: {e}")

        print("\n")  # Add space between specialties for readability

    return findings_by_display_name

def save_to_excel(findings_data, filename="nnt_lrs.xlsx", blank_values=False):
    """
    Saves likelihood ratios to an Excel file with each display_name as a separate sheet.
    If blank_values is True, the Likelihood Ratio column is left blank.
    The first row contains the full display_name, and column headers start from the second row.
    """
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        for display_name, findings in findings_data.items():
            if findings:
                # Prepare DataFrame
                df = pd.DataFrame(findings, columns=["Finding", "Likelihood Ratio"])

                if blank_values:
                    df["Likelihood Ratio"] = ""  # Clear likelihood ratio values

                # Insert full display_name as the first row
                full_name_row = pd.DataFrame({df.columns[0]: [display_name], df.columns[1]: [""]})
                df = pd.concat([full_name_row, df], ignore_index=True)

                # Save to Excel with sheet name as the **last** 31 characters
                sheet_name = display_name[-31:]
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)  # No default header

            else:
                print(f"Skipping {display_name} (No data found).")

    print(f"\nLikelihood ratios saved to {filename}")

# Fetch specialties and links
specialty_links = get_specialty_links()
findings_data = fetch_webpages(specialty_links)

# Save normal file
save_to_excel(findings_data, "nnt_lrs.xlsx", blank_values=False)

# Save version with blank likelihood ratios
save_to_excel(findings_data, "nnt_lrs_sans_number.xlsx", blank_values=True)

### Convert LR's to numerical format

This block takes in the excel spreadsheet with raw data from theNNT.com ("nnt_lrs.xlsx") and creates a new spreadsheet ("nnt_lrs_processed.xlsx") with a third column that contains a numerical version of the second column (raw data from theNNT) to be used as the LR_llm.


It removes any 'x's from the input, then determines whether the cell reports 

1. point estimate only (in which case use the point estimate)
2. point estimate + range (in which case take the point estimate), or 
3. range only (in which case, calculate the geometric mean)

It also counts the number of conditions (last 31 letters due to excel limitation) and LRs

In [ ]:
def parse_lr(lr_str):
    """
    Given a string from the 'Likelihood Ratio' cell, this function:
      - Removes any 'x' characters from the input.
      - If the string contains a parenthesized range (i.e. a point estimate plus a range),
        it returns the point estimate.
      - If the entire string is a range (e.g. "0.92-1.1", "3.3 to 4.8", "4.8–7.6"),
        it computes and returns the geometric mean.
      - Otherwise, it returns a float based on the first number found.
      - If the value is missing or cannot be parsed, returns NaN.
    """
    # Remove all 'x' characters and trim whitespace
    lr_str = lr_str.replace("x", "").strip()
    if lr_str == "":
        return np.nan

    # If parentheses exist, assume format "point_estimate (range)" and use the point estimate.
    if "(" in lr_str:
        point_part = lr_str.split("(")[0].strip()
        try:
            return float(point_part)
        except Exception:
            pass

    # Check for a range-only pattern.
    # This regex looks for two numbers separated by "to", "-" or "–" with optional whitespace.
    range_only_match = re.match(r'^\s*([0-9]*\.?[0-9]+)\s*(to|[-–])\s*([0-9]*\.?[0-9]+)\s*$', lr_str)
    if range_only_match:
        try:
            low = float(range_only_match.group(1))
            high = float(range_only_match.group(3))
            return math.sqrt(low * high)
        except Exception:
            return np.nan

    # Fallback: if no range-only pattern is found, extract the first number and return it.
    numbers = re.findall(r'([0-9]*\.?[0-9]+)', lr_str)
    if numbers:
        try:
            return float(numbers[0])
        except Exception:
            return np.nan

    return np.nan

# Load the original Excel file (each sheet has no header and the first row is the display name row)
input_filename = "nnt_lrs.xlsx"
output_filename = "nnt_lrs_processed.xlsx"

# Read all sheets from the Excel file into a dictionary of DataFrames.
excel_sheets = pd.read_excel(input_filename, sheet_name=None, header=None)

total_lr_count = 0
sheet_counts = {}

with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    for sheet_name, df in excel_sheets.items():
        numerical_lr = []
        # Process each row in the sheet.
        for idx, row in df.iterrows():
            # For the header row (assumed to be the first row: condition label), add an empty string.
            if idx == 0:
                numerical_lr.append("")
            else:
                cell_value = row[1]  # The original "Likelihood Ratio" is in the second column (index 1)
                if pd.isna(cell_value) or str(cell_value).strip() == "":
                    numerical_lr.append("")
                else:
                    numerical_lr.append(parse_lr(str(cell_value)))
        
        # Insert the new column immediately after the "Likelihood Ratio" column.
        # This makes the new column the third column.
        df.insert(2, "Numerical LR", numerical_lr)
        
        # Remove rows (except the header) where the new "Numerical LR" is empty or NaN.
        header = df.iloc[[0]]  # Keep the header row (the condition label)
        data = df.iloc[1:]
        data = data[data["Numerical LR"].apply(lambda x: not (x == "" or pd.isna(x)))]
        filtered_df = pd.concat([header, data], ignore_index=True)
        
        # Insert a new row (after the condition label row) with the column labels.
        # The final sheet will have:
        #   Row 0: Condition label (from the original sheet)
        #   Row 1: Column labels: 'finding', 'lr_raw', and 'lr_num'
        #   Row 2+: Data rows
        col_labels = pd.DataFrame([["finding", "lr_raw", "lr_reported"]], columns=filtered_df.columns)
        final_df = pd.concat([filtered_df.iloc[[0]], col_labels, filtered_df.iloc[1:]], ignore_index=True)
        
        # Count the number of LR values for this sheet (exclude the two header rows).
        lr_count = len(final_df) - 2
        sheet_counts[sheet_name] = lr_count
        total_lr_count += lr_count
        
        # Write the modified DataFrame to the new Excel file.
        # The output maintains the original format: no index and no additional header row.
        final_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

# Display counts.
num_sheets = len(excel_sheets)
print(f"Processed {num_sheets} condition(s) (sheets).")
for sheet, count in sheet_counts.items():
    print(f"Sheet '{sheet}' has {count} LR value(s).")
print(f"Total LR values processed across all sheets: {total_lr_count}.")

print(f"Processed Excel file saved as '{output_filename}'")

### Estimate LRs

NOTE: for the real run at this, we'll want to do some manual editing of the info columns - as there are some where it is a lab value that references the preceeding value (not currently automated to account for)

This code block reads in the data from the nnt_lr_processed.xlsx excel file and calls a list of openAI models to have them give there best (single) estimate of the LR. Then, it rights a new spreadsheet nnt_lr_estimates that includes columns in each spreadsheet for each estimation. 

In [ ]:
# Define the response schema expecting a floating point number.
class LRResponse(BaseModel):
    value: float

def estimate_lr(diagnosis: str, info_val: str, client, model: str) -> float:
    """
    Calls the LLM with a prompt containing the diagnosis and a finding.
    Returns the estimated likelihood ratio as a floating point number.
    """
    lr_prompt = """You are an expert in medical diagnosis who is giving assessments of how important a piece of information is when determining whether a patient has a particularly condition. Your task is to estimate the likelihood ratio of a finding for a disease. Recall that the likelihood ratio represents how much the ratio between the odds of disease given a result for a lab value, whether a physical exam finding is present, or whether a comorbidity is present over the odds of disease when you did not know the result.
You will receive inputs in the following format; Target condition: <Condition, e.g. Cardiac chest pain>. Finding: <piece of information, e.g. ‘No radiation to the neck, arm, or jaw’>.
So, for example. If the odds of a Condition Z being present was 1 (meaning 50% probability) before we knew anything, but then we got a result (Finding A) it became 2 (meaning 2:1 odds or 66% probability), then the likelihood ratio would be 2. 
Given a condition and a finding, you will provide your best estimate of the likelihood ratio as a floating point number. Return your answer in valid JSON with the following schema: { 'value': <floating point number greater than 0> }.\n\n

Remember, stronger evidence in favor of a condition has a value farther above 1. Strong evidence against a diagnosis has a value farther below 1 (closer to 0). A likelihood ratio of 10 is equally strong evidence for a condition as a likelihood ratio of 0.1 is against it. Likelihood ratios near 1 represent weak evidence for or against. 

Here is how I would like you to approach the problem:
First, consider the condition you are predicting (Condition: ___). Is the condition a medical diagnosis? If so, what kind of findings are usually present in someone who has that condition. Does the condition specify a certain type of patient? If so, how does that change things? 
Then, consider the finding. If a finding is much more common among patients who have the condition of interest than among patients who do not have the condition of interest, then the likelihood ratio should be high. This might be because the finding is a consequence of the disease, indicates that an enabling condition is present, indicates that a frequently comorbid condition is present, or is related to the pathology of the condition. In general, likelihood ratios over about 20 are pathognomonic, above 5 or so is extremely strong evidence in favor, above 2.5 or so is strong evidence, above 1.4 is so-so evidence, and 1-1.4 is pretty weak evidence. Conversely, if the finding is more common in people who do NOT have the condition, then the likelihood ratio should be below 0. Similarly, a likelihood ratio below 0.05 would exclude the condition in most situations, below 0.2 would be extremely strong evidence against, below 0.4 would be strong evidence against, below 0.71 is so-so, and between 0.71 and 1 is pretty weak evidence against (meaning, it just doesn’t change the odds of the condition much). 

Here are some hypothetical examples to consider: 
    Prompt = Target condition: Cardiac Chest Pain. Finding: Pain not worse with exertion (requires they clarify exercise 1hr after meal).
    You would reason that because cardiac chest pain is usually worse with exertion because exertion worsens cardiac demand for oxygen, and thus worsens ischemia.
    Response = {
        ‘value’: 0.4
    }

    Prompt =  Target condition: Cardiac Chest Pain. Finding: No tobacco.
    You would reason that because being someone who smokes increases your risk of coronary artery disease, and thus being a never smoker means you’re at less risk… but many people who have heart attacks still smoke, so it’s only a weak predictor. 
    Response = {
        ‘value’: 0.75
    }

    Prompt = Target condition: Cardaic Chest Pain. Finding = enjoys playing chess.
    You would reason that because enjoying chest has no relationship to having a heart attack.
    Response = {
        ‘value’: 1
    }

    Prompt = Target condition: Cardiac Chest Pain. Finding = pain located behind the sternum
    You would reason that because cardiac chest pain is often experienced behind the sternum (thus, more likely), but so are many other causes of chest pain - like GERD.
    Response = {
        ‘value’: 1.2
    }

    Prompt = Condition: Cardiac Chest Pain. Finding = pain worse with exertion.
    You would reason that because the increased myocardial oxygen consumption worsens the pain if oxygen delivery to the myocardium is the cause, as it is in heart attacks.
    Response = {
        ‘value’: 3.4
    }

    OK: here’s the prompt.. """
    
    messages = [
        {"role": "system", "content": lr_prompt},
        {"role": "user", "content": f"Condition: {diagnosis}\nFinding: {info_val}"}
    ]
    
    # Call the LLM using the provided model name.
    completion = client.beta.chat.completions.parse(
        model=model,  # use the current model from the list
        messages=messages,
        response_format=LRResponse,
    )
    
    # Extract and return the floating point estimate.
    lr_response = completion.choices[0].message.parsed
    return lr_response.value

# Initialize the OpenAI (or your chosen) client using your API key.
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# List of model names to iterate over.
model_names = ['gpt-4o-mini-2024-07-18', 'gpt-4o-2024-08-06'] 
# 'o1-mini-2024-09-12' doesn't support structured calls yet.
# don't have access to o1 or o3-mini yet thorugh api


# Read the processed Excel file.
# We use header=None so that row 0 (the diagnosis row) and row 1 (the column headers row) are preserved.
input_filename = "nnt_lrs_processed.xlsx"
sheets = pd.read_excel(input_filename, sheet_name=None, header=None)

# Process each sheet
for sheet_name, df in sheets.items():
    # Set diagnosis from the first row (row index 0, first cell)
    diagnosis = df.iloc[0, 0]
    
    # For each model in the list, call the LLM and add a new column with the estimation.
    for model in model_names:
        new_col_header = "lr_" + model
        new_col = []  # This list will hold one value per row
        
        # Iterate over each row in the sheet.
        # Row 0 is the diagnosis row; row 1 is the existing column labels.
        for i in range(len(df)):
            if i == 0:
                new_col.append("")  # Leave the diagnosis row unchanged.
            elif i == 1:
                new_col.append(new_col_header)  # Insert the new column header in row 1.
            else:
                # For data rows, use the "finding" from the first column (index 0)
                info_val = df.iloc[i, 0]
                try:
                    estimated_lr = estimate_lr(diagnosis, info_val, client, model)
                except Exception as e:
                    estimated_lr = "ERROR"
                    print(f"Error estimating LR for sheet '{sheet_name}', row {i}, model {model}: {e}")
                new_col.append(estimated_lr)
        
        # Insert the new column at the end of the dataframe.
        df.insert(df.shape[1], new_col_header, new_col)
    
    # Update the sheet data in our dictionary.
    sheets[sheet_name] = df

# Write out the modified sheets to a new Excel file.
output_filename = "nnt_lrs_with_estimated.xlsx"
with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    for sheet_name, df in sheets.items():
        # Write without adding pandas default headers or indices.
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print(f"Processed Excel file saved as '{output_filename}'")